In [29]:
from copy import deepcopy
from collections import defaultdict
import random

## Reading and initializing functions

In [62]:
def get_sudokus(filename, sudokusize=9):
    '''
    get_states is to read multiple sudokus in the format:
    ..8..4..5...64..5 etc.
    returns a list of sudoku board states as clauses in a dict
    '''
    sudokus = []
    with open(filename) as sudoku_states:
        for i, state in enumerate(sudoku_states):
            sudoku = {}
            pointers = {}
            row = 1
            index = 0
            for j, content in enumerate(state):
                if j%sudokusize == 0 and j > 0:
                    row+=1
                try:
                    int(content)
                    literal = int(str(row) + str(j%sudokusize+1) + content)
                    sudoku[index] = [literal]
                    if literal not in pointers.keys():
                        pointers[literal] = [index]
                    else:
                        pointers[literal].append(index)
                    index += 1
                except:
                    # is not convertible to int
                    continue
            sudokus.append((sudoku, pointers))
    return sudokus

def get_constraints_from_dimacs(filename, prevConstraints={}, prevPointers={}):
    '''
    Convert dimacs file to a dictionary of constraints
    and pointers. Allows merging with previous pointers and constraints.
    '''
    constraints = prevConstraints # dictionary with constraints as: clausenumber->[literals]
    pointers = prevPointers # dictionary with pointers to clause numbers for literals as: literals->[clausenumbers]
    initial_length = len(constraints)
    with open(filename) as f:
        for i, clause in enumerate(f):
            index = i + initial_length
            if clause[0] == 'c' or clause[0] == 'p':
                # skip comments etc.
                continue
            constraints[index]=[]
            for literal in clause.split():
                literal = int(literal)
                if literal !=0:
                    constraints[index].append(literal)
                    if literal not in pointers.keys():
                        pointers[literal]=[index]
                    else:
                        pointers[literal].append(index)
                    
    return constraints, pointers

    

## DP helper functions for constraints updates

In [31]:
def assign_literal(constraints, pointers, literal, assignments):
    '''
    assign the given literal to be True.
    assign_literal is aimed to be one recursion stack, so deepcopies are made
    in order to properly backtrack
    '''
    constraintsCopied = deepcopy(constraints)
    pointersCopied = deepcopy(pointers)
    assignmentsCopied = deepcopy(assignments)
    assignmentsUpdated = update_assigments(assignmentsCopied, literal)
    constraintsUpdated, pointersUpdated = update_constraints(constraintsCopied, pointersCopied, literal)
    return constraintsUpdated, pointersUpdated, assignmentsUpdated
     
def update_constraints(constraints, pointers, literal):
    '''
    literal: e.g. 112 or 112'
    Updates constraints and pointers by removing all clauses 
    with the given literal, because the clause becomes true.
    Remove counter-literal out of all clauses.
    '''
    ## Find clauses with literal and remove them
    clauses = list(pointers[literal])
    for clause in clauses:
        for lit in constraints[clause]:
            pointers[lit].remove(clause)
        del constraints[clause]
    del pointers[literal]
    
    ## Find clauses with counter-literal and remove its counter-literal
    counterLiteral = -literal
    clauses = list(pointers[counterLiteral])
    del pointers[counterLiteral]
    for clause in clauses:
        constraints[clause].remove(counterLiteral)

    return constraints, pointers

def update_assigments(assignments, literal):
    '''
    Keep track of assigned literals.
    Needed in order to return satisfiable solution of literals.
    '''
    if literal in assignments:
        print("already assigned", literal, "(in this stack), returning...")
        return
    else:
        assignments.append(literal)
        return assignments
    
def check_tautologies(constraints, pointers):
    '''
    Check for clauses with a tautology and remove
    them.
    '''
    foundOne = False
    for clause in constraints.keys():
        literals = constraints[clause]
        counterLiterals = [-x for x in literals]
        for i in literals:
            for j in counterLiterals:
                if i == j:
                    del constraints[clause]
                    pointers[i].remove(clause)
                    foundOne = True
                    break
    if foundOne:
        print("Found tautologies and removed corresponding clauses")
    else:
        print("No tautologies found")
                  
    return constraints, pointers  

## DP cases

In [32]:
def empty_clause(constraints):
    for clause in constraints.keys():
        if not constraints[clause]:
            return True
        
    return False

def unit_clause(constraints): 
    for clause in constraints.keys():
        if len(constraints[clause])==1:
            literal = constraints[clause][0]
            return literal

    return False

def unit_propagate(constraints, pointers, assignments):
    '''
    Automatically assign the literal of all unit clauses
    until there are no unit clauses left.
    '''
    literal = unit_clause(constraints)
    while(not empty_clause(constraints) and literal):
        constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments)
        literal = unit_clause(constraints)
    return constraints, pointers, assignments

def pure_literal(literals):
    for literal in literals:
        if -literal not in literals:
            return literal
        
    return False

def random_literal(literals): 
    literal = random.choice(literals) 
    return literal

## DP main function

In [33]:
def DP(constraints, pointers, assignments): 
    '''
    DP main recursive function. Always returns satisfiable
    assignment for all constraints if there is one.
    '''
    constraints, pointers, assignments = unit_propagate(constraints, pointers, assignments)
    
    if len(constraints)==0:
        return assignments
   
    if empty_clause(constraints):
        return False
    
    all_literals = list(pointers.keys())
    
    literal = pure_literal(all_literals)
    if literal:
        constraints, pointers, assignments = assign_literal(constraints, pointers, literal, assignments)
        DP(constraints, pointers, assignments)
    
    literal = random_literal(all_literals)
    constraintsUpdated, pointersUpdated, assignmentsUpdated = assign_literal(constraints, pointers, literal, assignments)
    assignmentsFinal = DP(constraintsUpdated, pointersUpdated, assignmentsUpdated)
    if assignmentsFinal:
        return assignmentsFinal
    else:
        constraints, pointers, assignments = assign_literal(constraints, pointers, -literal, assignments)
        return DP(constraints, pointers, assignments)

In [63]:
def set_up():
    ## Get board and rules
    print("Reading and analyzing...")
    constraints, pointers = get_sudokus('1000 sudokus.txt')[10]
#     print(constraints, pointers)
#     constraints, pointers = get_constraints_from_dimacs('sudoku-example.txt')
    constraints, pointers = get_constraints_from_dimacs('sudoku-rules.txt', constraints, pointers)
    constraints, pointers = check_tautologies(constraints, pointers)
    
    ## Run DP
    print("Starting DP algorithm...")
    assignments = DP(constraints, pointers, [])
    if assignments:
        print("Solution found")
        boardPositions = [a for a in assignments if a > 0]
        print(boardPositions)
    else:
        print("No solution found")
        
set_up()

Reading and analyzing...
No tautologies found
Starting DP algorithm...
Solution found
[134, 152, 183, 248, 269, 377, 425, 453, 467, 498, 595, 624, 639, 656, 681, 715, 826, 838, 937, 954, 979, 991, 814, 729, 731, 757, 147, 558, 618, 697, 673, 859, 887, 441, 549, 564, 489, 474, 861, 344, 363, 166, 175, 271, 388, 199, 128, 111, 319, 255, 351, 284, 335, 322, 292, 396, 521, 517, 533, 227, 923, 912, 416, 432, 213, 236, 872, 893, 985, 743, 946, 762, 845, 968, 642, 665, 576, 582, 778, 786, 794]
